In [6]:

import json
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow import keras


from sensorflow_model.dataset import get_dataset 


In [7]:

def normalize_with_saved_stats(X, stats_path="normalization.json"):
    with open(stats_path, "r") as f:
        stats = json.load(f)

    mean = np.array(stats["mean"])
    std = np.array(stats["std"])
    X_flat = X.reshape(-1, X.shape[-1])
    X_scaled = (X_flat - mean) / std
    return X_scaled.reshape(X.shape)


In [8]:


def load_test_dataset(folder_path):
    X_all, y_all = [], []

    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            filepath = os.path.join(folder_path, file)

            # window_size and stride same as training
            X, y = get_dataset(filepath, window_size=20, stride=5, normalize=True)
            X_all.append(X)
            y_all.append(y)

    return np.concatenate(X_all), np.concatenate(y_all)


In [10]:

# Step 1: Load model
model = keras.models.load_model("../final_model.keras")


c:\Users\mohit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 46 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [13]:

# Step 2: Load test data
x_test, y_test = load_test_dataset("../TestData/with")


In [14]:

# Step 3: Evaluate
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
print(f"\n Test Accuracy: {accuracy * 100:.2f}%")

39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.2197 - loss: 5.1468

 Test Accuracy: 24.86%
